Hopefully, This script will allow me to load up and fix up the moored data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pyproj
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict
from matplotlib.colors import LogNorm

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Markdown, display

%matplotlib inline

In [3]:
saveloc='/ocean/kflanaga/MEOPAR/savedData/phytoData'
year=2018
Chl_N=1.8

In [4]:
df = pd.read_csv(f'{saveloc}/KingCounty_Marine_Phytoplankton_Samples_by_Taxonomic_Group.csv')

In [5]:
df.columns

Index(['Locator', 'Site Name', 'Lon', 'Lat', 'Sample Number', 'Collect Date',
       'Depth (m)', 'Taxonomic Group', 'Taxon', 'Rank', 'Kingdom', 'Phylum',
       'Class', 'Order', 'Family', 'Genus', 'Species', 'Biovolume (mm^3/L)',
       'Abundance (particles/mL)', 'Quality', 'Steward Note', 'Sample ID',
       'Grab ID', 'Profile ID', 'Replicates', 'Replicates Of', 'DataSource'],
      dtype='object')

In [6]:
df['Taxonomic Group'].unique()

array(['Centric diatoms', 'dinoflagellates', 'Amylax triacantha',
       'Pennate diatoms', 'Ciliates', 'Dinoflagellates',
       'Silicoflagellates', 'Centric diatom', 'Flagellates', 'Diatoms',
       'Brown algae', 'Haptophytes', 'Unidentified ',
       'Unidentified dinoflagellates', 'Unidentified zooplankton'],
      dtype=object)

In [7]:
north_east_dict={'JSUR01':(287580, 1250910),
'KSBP01': (275439, 1248062),
'KSSK02': (245121, 1242740),
'LTED04': (223909, 1264675),
'LSEP01': (223360, 1247399),
'LSKQ06': (212065, 1248334),
'LSNT01': (198653, 1245194),
'MSJN02': (169328, 1244585),
'NSEX01': (134701, 1255331),
'NSAJ02': (140223, 1239011)}

In [8]:
northing=[]
easting=[]
for i in df.index:
    for j in north_east_dict:
        if df.iloc[i]['Locator'] == j:
            nor,eas = north_east_dict[j]
            northing.append(nor)
            easting.append(eas)

In [9]:
df['Easting']=easting
df['Northing']=northing

In [43]:
#The correct epsg was found by plugging in NAD83 Washington into the Search bar 
#of the Spatial reference website and finding the epsg value of Northern Washington
transformer = pyproj.Transformer.from_crs("epsg:3690", 'epsg:4326')

In [42]:
transformer.transform(xx=easting[1],yy=northing[1])

(47.77739788274532, -122.41761158500448)

In [33]:
for i in df.index:
    lat,lon=transformer.transform(xx=df['Easting'][i],yy=df['Northing'][i])
    df.loc[i, 'Lat']=lat
    df.loc[i, 'Lon']=lon

In [34]:
df

,Locator,Site Name,Lon,Lat,Sample Number,Collect Date,Depth (m),Taxonomic Group,Taxon,Rank,...,Quality,Steward Note,Sample ID,Grab ID,Profile ID,Replicates,Replicates Of,DataSource,Easting,Northing
0,JSUR01,Point Wells,-122.417612,47.777398,L68492-2,09/05/2017,1.10,Centric diatoms,Actinoptychus senarius,Species,...,1,NaN,36197,36197.0,29122,NaN,NaN,KCEL,1250910,287580
1,JSUR01,Point Wells,-122.417612,47.777398,L69494-2,03/07/2018,1.00,Centric diatoms,Actinoptychus senarius,Species,...,1,NaN,154747,154747.0,67895,NaN,NaN,KCEL,1250910,287580
2,JSUR01,Point Wells,-122.417612,47.777398,L72509-2,06/17/2019,1.00,Centric diatoms,Actinoptychus senarius,Species,...,1,NaN,172548,172548.0,75611,NaN,NaN,KCEL,1250910,287580
3,JSUR01,Point Wells,-122.417612,47.777398,L62667-3,05/18/2015,0.99,Centric diatoms,Actinoptychus senarius,Species,...,2,NaN,528,528.0,14,NaN,NaN,KCEL,1250910,287580
4,JSUR01,Point Wells,-122.417612,47.777398,L72418-2,06/03/2019,1.10,Centric diatoms,Actinoptychus senarius,Species,...,1,NaN,172539,172539.0,75608,NaN,NaN,KCEL,1250910,287580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23630,NSEX01,East Passage,-122.387082,47.358617,L75380-9,09/09/2020,0.99,Unidentified zooplankton,unidentified zooplankton,NaN,...,1,NaN,193159,193159.0,80651,NaN,NaN,KCEL,1255331,134701
23631,NSEX01,East Passage,-122.387082,47.358617,L75498-9 DUP,09/22/2020,1.00,Unidentified zooplankton,unidentified zooplankton,NaN,...,1,NaN,193906,193719.0,80813,NaN,NaN,KCEL,1255331,134701
23632,NSEX01,East Passage,-122.387082,47.358617,L75498-9,09/22/2020,1.00,Unidentified zooplankton,unidentified zooplankton,NaN,...,1,NaN,193893,193719.0,80813,NaN,NaN,KCEL,1255331,134701
23633,NSEX01,East Passage,-122.387082,47.358617,L75813-9,11/03/2020,1.00,Unidentified zooplankton,unidentified zooplankton,NaN,...,1,NaN,193922,193922.0,80982,NaN,NaN,KCEL,1255331,134701


In [40]:
##### Saving data as Pickle files to be used in the summary file
#saveloc='/ocean/kflanaga/MEOPAR/mooredData'
#with open(os.path.join(saveloc,f'data_{Mooring}_{year}.pkl'),'wb') as hh:
#    pickle.dump(df,hh)
    
#with open(os.path.join(saveloc,f'daily_data_{Mooring}_{year}.pkl'),'wb') as hh:
#    pickle.dump(df_daily_avg,hh)